# Single crystals in Julia

Content of this notebook:
* How to create single crystal unitcells and supercells using `ase`'s algorithm
* How to visualize a crystal
* How to test a crystal against a reference from the Pyhton module `ase`

In [ ]:
import Pkg
Pkg.activate(".")
Pkg.instantiate()

In [ ]:
# Pkg.add("LaTeXStrings")

In [ ]:
using Plots
using LaTeXStrings
using JSON
using LinearAlgebra
using Test
using SingleCrystal

## Specifying single crystals

In [ ]:
crystal_specs = Dict(
    # NaCl structure
    "NaCl" => Dict(
        "symbols" => ["Na", "Cl"],
        "basis" => [[0. 0. 0.], [.5 .5 .5]], # scaled coordinates
        "nr" => 225,
        "setting" => 1,
        "a_direction" => [1.; 0.; 0.],
        "ab_normal" => [0.; 0.; 1.],
        "cellpar" => [5.64, 5.64, 5.64, 90, 90, 90]
    ),
    # Al fcc structure
    "Al_fcc" => Dict(
        "symbols" => ["Al"],
        "basis" => [[0. 0. 0.],], # scaled coordinates
        "nr" => 225,
        "setting" => 1,
        "a_direction" => [1.; 0.; 0.],
        "ab_normal" => [0.; 0.; 1.],
        "cellpar" => [4.05, 4.05, 4.05, 90, 90, 90]
    ),
    # Fe bcc structure
    "Fe_bcc" => Dict(
        "symbols" => ["Fe"],
        "basis" => [[0. 0. 0.],], # scaled coordinates
        "nr" => 229,
        "setting" => 1,
        "a_direction" => [1.; 0.; 0.],
        "ab_normal" => [0.; 0.; 1.],
        "cellpar" => [2.87, 2.87, 2.87, 90, 90, 90]
    ),
    # Mg hcp structure
    "Mg_hcp" => Dict(
        "symbols" => ["Mg"],
        "basis" => [[1/3 2/3 3/4],], # scaled coordinates
        "nr" => 194,
        "setting" => 1,
        "a_direction" => [1.; 0.; 0.],
        "ab_normal" => [0.; 0.; 1.],
        "cellpar" => [3.21, 3.21, 5.21, 90, 90, 120]
    ),
    # Diamond structure
    "Diamond" => Dict(
        "symbols" => ["C"],
        "basis" => [[0. 0. 0.],], # scaled coordinates
        "nr" => 227,
        "setting" => 1,
        "a_direction" => [1.; 0.; 0.],
        "ab_normal" => [0.; 0.; 1.],
        "cellpar" => [3.57, 3.57, 3.57, 90, 90, 90]
    ),
    # Rutile structure
    "Rutile" => Dict(
        "symbols" => ["Ti", "O"],
        "basis" => [[0. 0. 0.], [.3 .3 0.]], # scaled coordinates
        "nr" => 136,
        "setting" => 1,
        "a_direction" => [1.; 0.; 0.],
        "ab_normal" => [0.; 0.; 1.],
        "cellpar" => [4.6, 4.6, 2.95, 90, 90, 90]
    ),
    # CoSb3 skudderudite
    "Skudderudite" => Dict(
        "symbols" => ["Co", "Sb"],
        "basis" => [[.25 .25 .25], [0. .335 .158]], # scaled coordinates
        "nr" => 204,
        "setting" => 1,
        "a_direction" => [1.; 0.; 0.],
        "ab_normal" => [0.; 0.; 1.],
        "cellpar" => [9.04, 9.04, 9.04, 90, 90, 90]
    )
);

## Constructing a single crystal

### How to

#### Cubic BCC crystals using `SingleCrystal.make_bcc_unitcell`

`SingleCrystal.make_bcc_unitcell` one can directly create bcc unit cells with little effort.

In [1]:
SingleCrystal.make_bcc_unitcell("Fe", 3.4)

LoadError: UndefVarError: SingleCrystal not defined

In case you need fcc unit cells you can similarly make use of `SingleCrystal.make_fcc_unitcell`.

#### Cubic BCC crystal using `SingleCrystal.make_unitcell`

What `SingleCrystal.make_bcc_unitcell` does is call `SingleCrystal.make_unitcell` with certain parameters like

In [ ]:
symbols = ["Fe"] # chemical elements
basis = [[0. 0. 0.],] # scaled coordinates
nr = 229 # space group
setting = 1 # space group settig (greetings from ase)
a_direction = [1.; 0.; 0.] # direction of the first cell vector
ab_normal = [0.; 0.; 1.] # cross product of the first two cell vectors
cellpar = [2.87, 2.87, 2.87, 90, 90, 90]; # a, b, c, α, β, γ

In [ ]:
crystal = SingleCrystal.make_unitcell(basis, symbols, nr, setting, cellpar,
                       a_direction=a_direction, ab_normal=ab_normal)
println(crystal)

#### Skudderudite using `SingleCrystal.make_unitcell`

Alternatively you can create more complex structures, like Skudderudite

In [ ]:
name = "Skudderudite"

nr = crystal_specs[name]["nr"]
setting = crystal_specs[name]["setting"]
basis = crystal_specs[name]["basis"]
symbols = crystal_specs[name]["symbols"]
a_direction = crystal_specs[name]["a_direction"]
ab_normal = crystal_specs[name]["ab_normal"]
cellpar = crystal_specs[name]["cellpar"];

also using the `SingleCrystal.make_unitcell` convenience function

In [2]:
crystal = SingleCrystal.make_unitcell(basis, symbols, nr, setting, cellpar,
                       a_direction=a_direction, ab_normal=ab_normal)
println(crystal)

LoadError: UndefVarError: a_direction not defined

#### Plotting `crystal`

In [ ]:
function plot_crystal(cell::SingleCrystal.Cell;
        default_color::String="blue",
        element_color_map::Dict=Dict{String,String}("Fe" => "blue"),
        default_size::T=50,
        element_size_map::Dict=Dict{String,Any}()
    ) where T <: Real
    
    atoms, coords = cell.atoms, cell.positions 
    
    elements = Set([atom.name for atom in atoms])
    for element in elements
        if !haskey(element_color_map, element)
            element_color_map[element] = default_color
        end
        if !haskey(element_size_map, element)
            element_size_map[element] = default_size
        end
    end
    colors = [element_color_map[atom.name] for atom in atoms]
    sizes = [element_size_map[atom.name] for atom in atoms]

    x = [v[1] for v in coords]
    y = [v[2] for v in coords]
    z = [v[3] for v in coords]
    return @gif for i in range(0, stop=2π, length=100)
        scatter(x, y, z, camera=(10*(1+cos(i)),5),
            markersize=sizes, legend=false, 
            color=colors, aspect_ratio=:equal,
            xlabel=L"x", ylabel=L"y", zlabel=L"z",
            title=string(length(atoms), " atoms of: ", join(elements, ","))
        )
    end
end

In [ ]:
element_color_map = Dict("Na" => "purple", "Cl" => "green",
    "Co"=>"pink", "Sb"=>"purple")
element_size_map = Dict("Na"=>20,"Cl"=>10, "Co"=>15, "Sb"=>15)

In [ ]:
plot_crystal(crystal, 
    element_color_map=element_color_map,
    element_size_map=element_size_map,
    default_size=5)

#### Generating a supercell

By cloning and shifting the atoms of the unitcell along the cell box vectors by integers up to `nx`, `ny` and `nz` we create a *supercell*

In [ ]:
supercell = SingleCrystal.make_supercell(crystal, nx=3, ny=3, nz=3);

In [ ]:
plot_crystal(supercell, 
    element_color_map=element_color_map,
    element_size_map=element_size_map,
    default_size=5)

### Behind the scenes ops

Above the high level convenience functions were introduced to create unit cells. This section contains the behind the scenes in operations in `SingleCrystal.make_unitcell`. So, the following is only for you if you are curios, and not necessary knowledge to create unitcells with this package. 

#### Collecting symmetry operations

In [ ]:
spgs = SingleCrystal.load_spgs()
nr = crystal_specs[name]["nr"]
setting = crystal_specs[name]["setting"]
spg = spgs["$(nr): $(setting)"]

In [ ]:
function parse_spg(spg::Dict{String,Any})::Dict{String,Any}
    spg["subtrans"] = [Array{Float64}(reshape(v,3)) for v in spg["subtrans"]]
    spg["translations"] = [Array{Float64}(reshape(v,3)) for v in spg["translations"]]
    spg["rotations"] = [Array{Float64}(hcat(v...)) for v in spg["rotations"]]
    return spg
end

In [ ]:
spg = parse_spg(spg)

In [ ]:
function get_symops(spg::Dict)
    parities = spg["centrosymmetric"] ? [1,-1] : [1]
    symops = []
    @assert length(spg["rotations"]) == length(spg["translations"])
    for (parity, trans_sub) in Iterators.product(parities, spg["subtrans"])
        for (rot, trans) in Iterators.zip(spg["rotations"], spg["translations"])
            push!(symops, (parity * rot,
                           (trans + trans_sub) .% 1))
        end
    end
    return symops
end

In [ ]:
symops = get_symops(spg)
symops[:4]

#### Generating equivalent sites of the basis

In [ ]:
function fold(x::T) where T<:Any
    return x < 0 ? x + 1 : x
end

function get_equivalent_sites(basis::Array{Array{Float64,2},1}, symops)
    kinds, sites = [], []

    for (kind, pos) in enumerate(basis)
        for (rot, trans) in symops
            site = (transpose(pos * rot) + trans) .% 1
            site = fold.(site)
            isdifferent = !any([v ≈ site for v in sites])
            if ((length(sites) == 0) | isdifferent)
                push!(sites, site)
                append!(kinds, kind)
            end
        end
    end
    return sites, kinds
end

Using the basis and the symmetry operations we can generate the equivalent sites

In [ ]:
basis = crystal_specs[name]["basis"]
sites, kinds = get_equivalent_sites(basis, symops)

#### Computing the cell box vectors

In [ ]:
function make_unit_vec(x::Array{T,1}) where T<:Any
    return x / norm(x)
end

function get_coords(a_direction::Array{Float64,1}, ab_normal::Array{Float64,1})
    @assert dot(a_direction, ab_normal) ≈ 0.
    _x = make_unit_vec(a_direction)
    z = make_unit_vec(ab_normal)

    x = _x - dot(_x, ab_normal) * z
    xyz = hcat(x, cross(z,x), z)
    return xyz
end

In [ ]:
a_direction = crystal_specs[name]["a_direction"]
ab_normal = crystal_specs[name]["ab_normal"]
xyz = get_coords(a_direction, ab_normal)

In [ ]:
function deg2rad(x::T) where T <: Real
    return x * π / 180.
end

function get_cos(x::T) where T <: Real
    return x ≈ 90 ? 0 : cos(deg2rad(x))
end

function get_cell_vectors(cellpar::Array{Float64,1})
    a, b, c, α, β, γ = cellpar
    cos_α = get_cos(α)
    cos_β = get_cos(β)
    cos_γ = get_cos(abs(γ))
    sin_γ = abs(γ) ≈ 90 ? sign(γ) : sin(deg2rad(γ))
    cos_α, cos_β, cos_γ, sin_γ
    
    cy = (cos_α - cos_β * cos_γ) / sin_γ
    abc = hcat([a; 0; 0], b*[cos_γ; sin_γ; 0], c*[cos_β; cy; √(1-cos_β*cos_β-cy*cy)])
    return abc
end

In [ ]:
abc = get_cell_vectors(crystal_specs[name]["cellpar"])

In [ ]:
cell = abc * xyz

#### Storing everything within a `SingleCrystal.Cell` struct

In [ ]:
chemical_symbols, atomic_numbers, masses = SingleCrystal.get_chemical_info()

In [ ]:
# storing crystal properties in a `SingleCrystal.Cell` instance
el2atom_map = Dict(el => SingleCrystal.Atom(name=el, mass=masses[el]) for el in keys(masses))

cc = SingleCrystal.CartesianCoords(Float64)
box = SingleCrystal.PrimitiveVectors(cc, A₁=abc[:,1], A₂=abc[:,2], A₃=abc[:,3])

_spg = SingleCrystal.Spacegroup(nr, setting, kinds, sites)

crystal = SingleCrystal.Cell(
    [el2atom_map[symbols[v]] for v in kinds],
    [cell * v for v in  sites],
    box,
    [norm(abc[:,1]), norm(abc[:,2]), norm(abc[:,3])],
    _spg
)

## Sanity checking generated positions and cell boxes

Using `ase` as a reference we can directly check if our positions and cell boxes have the expected values. For this we load pre-computed values from disk.

In [ ]:
json_ase_crystals = SingleCrystal.load_refs()

### Checking a single crystal

In [ ]:
name = "Skudderudite"

In [ ]:
json_ase_crystal = json_ase_crystals[name]

In [ ]:
ase_crystal = SingleCrystal.parse_json_crystal(json_ase_crystal)

In [ ]:
function positions_match_ase(crystal::SingleCrystal.Cell, ase_crystal::Dict)
    return all([p0 ≈ p1 for (p0,p1) in zip(ase_crystal["positions"],crystal.positions)])
end

In [ ]:
@assert positions_match_ase(crystal, ase_crystal)

In [ ]:
function cell_matches_ase(crystal::SingleCrystal.Cell, ase_crystal::Dict)
    return crystal.box.M ≈ ase_crystal["cell"]
end

In [ ]:
@assert cell_matches_ase(crystal, ase_crystal)

### Looping all crystals

All right, looks good so far. Let's check all crystals

In [ ]:
crystals = Dict()

for name in keys(crystal_specs)
    @testset "$(name)" begin
        nr = crystal_specs[name]["nr"]
        setting = crystal_specs[name]["setting"]
        basis = crystal_specs[name]["basis"]
        symbols = crystal_specs[name]["symbols"]
        a_direction = crystal_specs[name]["a_direction"]
        ab_normal = crystal_specs[name]["ab_normal"]
        cellpar = crystal_specs[name]["cellpar"]

        crystal = SingleCrystal.make_unitcell(basis, symbols, nr, setting, cellpar,
                               a_direction=a_direction, ab_normal=ab_normal)
        ase_crystal = SingleCrystal.parse_json_crystal(json_ase_crystals[name])
        
        @testset "positions match" begin 
            @test positions_match_ase(crystal, ase_crystal)
        end
        @testset "cell match" begin
            @test cell_matches_ase(crystal, ase_crystal)
        end
        
        crystals[name] = crystal
        
    end
end

## Storing crystal as json readable by `ase.Atoms.fromdict`

Since one may want to re-use the crystals created with this julia package from python with ase, let's export them.

In [ ]:
ds = Dict(name => SingleCrystal.cell2dict(crystals[name])
          for name in keys(crystals))

open("julia-atoms.json","w") do f
    JSON.print(f, ds)
end